# Часть 1: Трасса вычислений и Расширение низкой степени

### Основа
Трасса представлений - это представление процесса вычислений, которые необходимо доказать. Оно содержит последовательность состояний, фиксирующих выполнение программы или алгоритма на каждом шаге.

Например, если программа имеет n количество шагов и переменных ($v_1, v_2, ..., v_m$), то Трассу (Trace) можно представить в виде таблицы, где каждая строка соответствует шагам вычислений, а столбцы – переменным (регистрам)

Вся трасса представляет собой матрицу $m \times n$, где $n$ – число шагов вычислений, а $m$ – число переменных/регистров

#### Как используется Trace?

1) Описание вычислений: 
    - Каждое состояние вычисления (шаг) описывается через определённый набор переменных.
    - Для каждого шага фиксируются значения этих переменных.
2) Кодирование как полином:
    - Значения трассы преобразуются в функции низкой степени, которые легко проверяются (см. Low-Degree Extension ниже).
    - Это позволяет преобразовать сложное вычисление в проверяемую математическую структуру.

Для представления элементов поля мы используем класс FieldElement.
Вы можете создавать экземпляры FieldElement из целых чисел, а затем складывать, умножать, делить, получать инверсию и так далее. Базовым полем этого класса является $F_{3221225473} = 3 \cdot 2^{30} + 1$, поэтому все операции выполняются по модулю 3221225473.

In [1]:
from fields.field import FieldElement
FieldElement(3221225472) + FieldElement(10)

9

##### FibonacciSq Trace

Для начала построим список a длины 1023, первыми двумя элементами которого будут объекты FieldElement, представляющие 1 и 3141592 соответственно. Следующие 1021 элемент будут последовательностью FibonacciSq, индуцированной этими двумя элементами. a называется следом FibonacciSq, или, когда контекст ясен, следом.
Исправьте приведенный ниже код, чтобы заполнить a:

In [2]:
a = [FieldElement(1), FieldElement(3141592)]
while len(a) < 1023:
    a.append(a[-2] * a[-2] + a[-1] * a[-1])

##### Выполним тестирование 
Запустите следующую ячейку, чтобы проверить правильность заполнения.
Обратите внимание, что на самом деле это верификатор, хотя и очень наивный и не очень понятный, поскольку он просматривает последовательность, элемент за элементом, убеждаясь в ее правильности.

In [3]:
assert len(a) == 1023, 'След должен состоять ровно из 1023 элементов.'
assert a[0] == FieldElement(1), 'Первый элемент в трассе должен быть элементом единицы.'
for i in range(2, 1023):
    assert a[i] == a[i - 1] * a[i - 1] + a[i - 2] * a[i - 2], f'Правило рекурсии FibonacciSq не применяется для индекса {i}'
assert a[1022] == FieldElement(2338775057), 'Неправильный последний элемент!'
print('Успешно!')

Успешно!


#### Low-Degree Extension (Расширение низкой степени)
Low-Degree Extension (LDE) — это процесс преобразования значений вычислений (Trace) в полиномы низкой степени. Это ключевой шаг, который делает проверку вычислений математически удобной и позволяет использовать свойства полиномов.

Основная идея LDE:
 - Трасса вычислений (Trace), заданная как конечное множество дискретных значений, интерполируется полиномами низкой степени.
 - Эти полиномы затем "расширяются" на большее множество точек, сохраняя свою низкую степень.

Почему это важно?
 - Полиномы низкой степени легко проверяются на равенство или корректность с использованием малого количества случайных выборок.
 - Расширение до большего множества точек делает доказательство более защищённым от подделок.

Теперь мы хотим представить последовательность как оценку некоторого, пока неизвестного, многочлена $f$ степени 1022 (в силу теоремы Унисолвенса). В качестве области выберем некоторую подгруппу $G \subseteq F^{\times}$ размером 1024, по причинам, которые станут понятны позже.

##### Найдем группу размером 1024
Если мы найдем элемент $g \in F$ чей (мультипликативный) порядок равен 1024, то 
 сгенерирует такую группу. Класс FieldElement предоставляет статический метод generator(), который возвращает элемент, генерирующий $F^{\times}$ ($|F^{\times}|$)

In [4]:
g = FieldElement.generator() ** (3 * 2 ** 20)
G = [g ** i for i in range(1024)]

In [5]:
# Проверяет правильность g и G.
assert g.is_order(1024), 'Генератор g имеет неправильный порядок.'
b = FieldElement(1)
for i in range(1023):
    assert b == G[i], 'i-е место в G не равно i-й степени g.'
    b = b * g
    assert b != FieldElement(1), f'g имеет порядок {i + 1}'
    
if b * g == FieldElement(1):
    print('Успешно!')
else:
    print('g имеет порядок > 1024')

Успешно!


##### Класс полиномов

We provide you with a class called Polynomial. The simplest way to construct a Polynomial is by using the variable $X$, которая представляет собой формальную переменную $x$

In [6]:
from polynomials.polynomial import X
# Многочлен 2x^2 + 1.
p = 2*X**2 + 1
# Оцените p в 2 балла:
print(p(2))
# Введите имя полинома в последней строке ячейки, чтобы отобразить его.
p

9


##### Интерполирование полинома

Наш модуль полинома предоставляет функцию интерполяции Лагранжа, аргументами которой являются:

- x_values: x-значения G, для которых известны значения полинома. [Список].
- y_values: соответствующие y-значения. [Список]

Она возвращает единственный экземпляр многочлена степени < len(x_values), который оценивает y_values[i] по x_values[i] для всех i.

Запустите следующую ячейку, чтобы получить помощь по функции interpolate_poly.



In [7]:
from polynomials.polynomial import interpolate_poly
# Обратите внимание, что выполнение interpolate_poly может занять до минуты.
f = interpolate_poly(G[:-1], a)
v = f(2)

100%|██████████| 1023/1023 [00:04<00:00, 211.20it/s]


In [8]:
assert v == FieldElement(1302089273)
print('Success!')

Success!


##### Оценка на более крупном участке

След, рассматриваемый как оценка многочлена $f$ на $G$ теперь можно расширить, оценивая $f$
 в более широкой области, создавая таким образом код коррекции ошибок Рида-Соломона.
 
##### Cosets

Для этого мы должны определить более широкую область, на которой $f$ будут оценены. Мы будем работать с доменом, который в 8 раз больше, чем $G$.
Естественный выбор для такой области - взять некоторую группу $H$ размера 8192 (который существует потому, что 8192 делит ($|F^{\times}|$) и сдвиньте его на генератор $F^{\times}$,  тем самым получая coset из $H$.



In [9]:
w = FieldElement.generator()
h = w ** ((2 ** 30 * 3) // 8192)
H = [h ** i for i in range(8192)]
eval_domain = [w * x for x in H]

In [10]:
from hashlib import sha256
assert len(set(eval_domain)) == len(eval_domain)
w = FieldElement.generator()
w_inv = w.inverse()
assert '55fe9505f35b6d77660537f6541d441ec1bd919d03901210384c6aa1da2682ce' == sha256(str(H[1]).encode()).hexdigest(),\
    'Список H неверен. H[1] должно быть h (т. е. генератор H).'
for i in range(8192):
    assert ((w_inv * eval_domain[1]) ** i) * w == eval_domain[i]
print('Успешно!')

Успешно!


##### Оценка
Пора использовать interpolate_poly и Polynomial.poly для оценки по косете. Обратите внимание, что в нашем Python-модуле это реализовано по-настоящему наивно, поэтому интерполяция может занять до минуты.
Действительно, интерполяция и оценка полинома трассы - один из самых трудоемких шагов в протоколе STARK, даже при использовании более эффективных методов (например, БПФ).

In [11]:
f = interpolate_poly(G[:-1], a)
f_eval = [f(d) for d in eval_domain]

100%|██████████| 1023/1023 [00:04<00:00, 209.91it/s]


In [12]:
# Проверка на предварительно вычисленный хэш.
from hashlib import sha256
from channels.channel import serialize
assert '1d357f674c27194715d1440f6a166e30855550cb8cb8efeb72827f6a1bf9b5bb' == sha256(serialize(f_eval).encode()).hexdigest()
print('Успешно!')

Успешно!


##### Обязательства

В качестве схемы обязательств мы будем использовать деревья Меркла на основе Sha256. Простая реализация этой схемы доступна вам в классе MerkleTree. Запустите следующую ячейку (в рамках данного руководства она также служит тестом на корректность всех вычислений, проведенных до сих пор):

In [13]:
from merkles.merkle import MerkleTree
f_merkle = MerkleTree(f_eval)
assert f_merkle.root == '6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04'
print('Успешно!')

Успешно!


##### Channel
Теоретически система доказательств STARK представляет собой протокол взаимодействия двух сторон - проверяющего и проверяемого. На практике мы преобразуем этот интерактивный протокол в неинтерактивное доказательство с помощью эвристики Фиата-Шамира. В этом уроке вы будете использовать класс Channel, который реализует это преобразование. Этот канал заменяет верификатор в том смысле, что проверяющий (который вы пишете) будет отправлять данные и получать случайные числа или случайные экземпляры FieldElement.

Этот простой фрагмент кода инстанцирует объект канала и отправляет ему корень вашего дерева Меркла. Позже объект канала можно будет вызвать для получения случайных чисел или случайных элементов поля.

In [14]:
from channels.channel import Channel
channel = Channel()
channel.send(f_merkle.root)

И наконец, вы можете получить доказательство на данный момент (т.е. все, что было передано в канале до определенного момента), распечатав член Channel.proof.

In [15]:
print(channel.proof)

['send:6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04']
